In [ ]:
#default_exp dalle

In [ ]:
!ln -s ../DALL_E/dall_e dall_e

ln: failed to create symbolic link 'dall_e/dall_e': File exists


In [ ]:
from PIL import Image,ImageFile
from pathlib import Path
from fastai.basics import *
from fastai.text.all import *
from transformers import ReformerModelWithLMHead, ReformerTokenizerFast,ReformerConfig
from fastai.callback.wandb import *
import wandb
from text2anime.data_loading import *
from text2anime.reformer import *
from dall_e import map_pixels, unmap_pixels, load_model
Image.LOAD_TRUNCATED_IMAGES=True
ImageFile.LOAD_TRUNCATED_IMAGES=True
import wandb

In [ ]:
wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marii (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

InvalidVersionSpec: Invalid version '2.3.0>=2.1.0': invalid character(s)



In [ ]:
path=Path('../danbooru2020')
image_path=path/'512px' #sample
meta=path/'meta'
bs=32

In [ ]:
#with open('named_tags.pkl','rb') as f: named_tags=pickle.load(f)
with open('id_tags.pkl','rb') as f: id_tags=pickle.load(f)
full_table = pd.read_pickle('fulll_table.pkl')

In [ ]:
dls=DALLE_dataloader(image_path,full_table,id_tags,max_len=1152,seq_len=64)
dls.train.bs=bs

These seem broken with mixed image text data

In [ ]:
# Use to have limited dataset size
# dls.train.idx_max=len(dls.train_ds)-1
# dls.valid.idx_max=len(dls.valid_ds)-1

In [ ]:
encoder_path=download_data("https://cdn.openai.com/dall-e/encoder.pkl","dalle_encoder.pkl")
decoder_path=download_data("https://cdn.openai.com/dall-e/decoder.pkl","dalle_decoder.pkl")
with open(encoder_path, 'rb') as f: encoder=torch.load(encoder_path, map_location=torch.device('cuda:0'))
#with open(decoder_path, 'rb') as f: decoder=torch.load(decoder_path, map_location=torch.device('cuda:0'))

In [ ]:
configuration = ReformerConfig(is_decoder=True,axial_pos_shape=(64,34))
reformer=ReformerModelWithLMHead(configuration)
reformer.reformer.embeddings.word_embeddings = nn.Embedding(2500+1+8192,256) #2500 for text, 1 for text-image seperator, and 8192 for image tokens
#need 1+8192 for image tokens + seperator

In [ ]:
class Combined_Model(nn.Module):
    def __init__(self,transformer,vae):
        super().__init__()
        self.transformer=transformer
        self.vae=vae
        self.transformer.lm_head=nn.Linear(512,2500+1+8192)
    def forward(self,img,text):
        with torch.no_grad():
            bs=img.shape[0]
            sep=torch.tensor([2500])[None].repeat(bs,1).cuda()
            img=TensorText(self.vae(img).argmax( axis=1).flatten(1)+2500+1)#2500 for text, 1 for text-image sep
            sl=text.shape[1]+img.shape[1]+1
            pad=64*34+1-sl#2240-sl #32*32+1+1153
            pad=torch.tensor([[0]]).repeat(bs,pad).cuda()
            text=torch.cat((pad,text,sep,img),1)
        return self.transformer(text[:,:-1])[0],text[:,1:]

In [ ]:
model=Combined_Model(reformer,encoder)

z = torch
print(z.shape,z)
z = F.one_hot(z, num_classes=enc.vocab_size).permute(0, 3, 1, 2).float()

x_stats = dec(z).float()
x_rec = unmap_pixels(torch.sigmoid(x_stats[:, :3]))
x_rec = T.ToPILImage(mode='RGB')(x_rec[0])

In [ ]:
dls.train.bs=bs
class DropOutput(Callback):
    def after_pred(self): 
        self.learn.yb= (self.pred[1],)
        self.learn.pred = self.pred[0]
        
cbs=[DropOutput(),GradientAccumulation(n_acc=1024),WandbCallback()]

In [ ]:
opt_func=partial(Lamb,mom=0.9,sqr_mom=0.96,eps=1e-08)

In [ ]:
learn=Learner(dls, model, loss_func=CrossEntropyLossFlat(),cbs=cbs, opt_func=opt_func, metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.show_training_loop()

Start Fit
   - before_fit     : [TrainEvalCallback, GradientAccumulation, MixedPrecision, Recorder, WandbCallback, ProgressCallback]
  Start Epoch Loop
     - before_epoch   : [Recorder, ProgressCallback]
    Start Train
       - before_train   : [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - before_batch   : [MixedPrecision]
         - after_pred     : [DropOutput, MixedPrecision]
         - after_loss     : [GradientAccumulation, MixedPrecision]
         - before_backward: [MixedPrecision]
         - before_step    : [GradientAccumulation, MixedPrecision]
         - after_step     : [MixedPrecision]
         - after_cancel_batch: []
         - after_batch    : [TrainEvalCallback, Recorder, WandbCallback, ProgressCallback]
      End Batch Loop
    End Train
     - after_cancel_train: [Recorder]
     - after_train    : [Recorder, ProgressCallback]
    Start Valid
       - before_validate: [TrainEvalCallback, Recorder, ProgressCallback]
      Start Bat

In [ ]:
Learner??

In [ ]:
#learn.lr_find()

In [ ]:
learn.fit_flat_cos(1,lr=0.001) #0.0135 0.012 0.01 diverges

WandbCallback requires use of "SaveModelCallback" to log best model


epoch,train_loss,valid_loss,accuracy,perplexity,time


/home/molly/miniconda3/envs/fastai/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
config.num_buckets is not set. Setting config.num_buckets to 64...
config.num_buckets is not set. Setting config.num_buckets to 64...
config.num_buckets is not set. Setting config.num_buckets to 64...
/home/molly/miniconda3/envs/fastai/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:617: UserWarning: Metadata Warning, tag 296 had too many entries: 2, expected 1
  warnings.warn(
/home/molly/miniconda3/envs/fastai/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:617: UserWarning: Metadata Warning, tag 296 had too many entrie

from torch.profiler import profile, record_function, ProfilerActivity

inputs=dls.one_batch()[0]

with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    with torch.no_grad():
        encoder(inputs)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))